<a href="https://colab.research.google.com/github/Jasonang-13/Scope-1-2_GHG_Calculator/blob/main/Scope_1_%26_2_GHG_Emission_Carbon_Calculator_Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# Combined GHG Emissions Calculator

# Emission factors from standard sources
EMISSION_FACTORS = {
    'electricity': {
        'Malaysia - Peninsular': 0.0007740,
        'Malaysia - Sabah': 0.0005250,
        'Malaysia - Sarawak': 0.0001990,
        'Indonesia': 0.0007784,
        'Sri Lanka': 0.0005033,
        'Bangladesh': 0.0005794,
        'Cambodia': 0.0003973,
        'Nepal': 0.0000019,
        'Myanmar': 0.0004423,
        'Pakistan': 0.0003686,
        'Philippines': 0.0007074,
        'Laos': 0.0003205,
        'US': 0.000385,
        'EU Average': 0.00023,
        'Global Average': 0.0004,
        'Custom': 0.0
    },
    'diesel': 0.0028995,
    'petrol': 0.0023025,
    'cng': 0.0019518,
    'lpg': 0.0015571,
    'octane': 0.0023537
}

# Empty template for user data
USER_DATA = {
    'company_name': '',
    'historical_data': {}
}

class GHGCalculator:
    def __init__(self):
        self.emission_factors = EMISSION_FACTORS
        self.user_data = USER_DATA

    def set_company_name(self, name):
        """Set the company name for the user"""
        self.user_data['company_name'] = name

    def add_historical_data(self, year, scope1, scope2):
        """Add historical emissions data for a specific year"""
        self.user_data['historical_data'][year] = {
            'scope1': scope1,
            'scope2': scope2
        }

    def calculate_scope1(self, diesel_liters, petrol_liters, fleet_diesel_liters,
                         fleet_petrol_liters, cng_m3=0, lpg_m3=0, octane_liters=0):
        """Calculate Scope 1 emissions in tCO2e"""
        emissions = 0
        emissions += diesel_liters * self.emission_factors['diesel']
        emissions += petrol_liters * self.emission_factors['petrol']
        emissions += fleet_diesel_liters * self.emission_factors['diesel']
        emissions += fleet_petrol_liters * self.emission_factors['petrol']
        if cng_m3 > 0:
            emissions += cng_m3 * self.emission_factors['cng']
        if lpg_m3 > 0:
            emissions += lpg_m3 * self.emission_factors['lpg']
        if octane_liters > 0:
            emissions += octane_liters * self.emission_factors['octane']
        return emissions

    def calculate_scope2(self, electricity_kwh, country):
        """Calculate Scope 2 emissions in tCO2e"""
        if country in self.emission_factors['electricity']:
            return electricity_kwh * self.emission_factors['electricity'][country]
        else:
            return electricity_kwh * self.emission_factors['electricity']['Global Average']

    def get_historical_data(self):
        """Get user's historical emissions data"""
        return self.user_data['historical_data']

    def predict_emissions(self, year):
        """Simple prediction based on historical trend"""
        data = self.get_historical_data()
        if not data or len(data) < 2:
            return None

        years = sorted(data.keys())
        scope1_changes = []
        scope2_changes = []

        for i in range(1, len(years)):
            prev_year = years[i-1]
            curr_year = years[i]
            scope1_change = (data[curr_year]['scope1'] - data[prev_year]['scope1']) / data[prev_year]['scope1']
            scope2_change = (data[curr_year]['scope2'] - data[prev_year]['scope2']) / data[prev_year]['scope2']
            scope1_changes.append(scope1_change)
            scope2_changes.append(scope2_change)

        avg_scope1_change = sum(scope1_changes) / len(scope1_changes)
        avg_scope2_change = sum(scope2_changes) / len(scope2_changes)

        latest_year = max(years)
        latest_data = data[latest_year]

        predicted_scope1 = latest_data['scope1'] * (1 + avg_scope1_change)
        predicted_scope2 = latest_data['scope2'] * (1 + avg_scope2_change)

        return {
            'scope1': predicted_scope1,
            'scope2': predicted_scope2
        }

def clear_screen():
    """Clear the terminal screen"""
    print("\n" * 20)

def main():
    calculator = GHGCalculator()

    clear_screen()
    print("===== GHG Emissions Calculator =====")
    print("Welcome to the Public GHG Emissions Calculator!")
    print("This tool helps you calculate and analyze Scope 1 and Scope 2 emissions.")
    print("Let's start by setting up your organization information.")

    company_name = input("\nEnter your organization name: ")
    calculator.set_company_name(company_name)

    add_historical = input("\nDo you want to add historical emissions data? (y/n): ").lower() == 'y'
    if add_historical:
        try:
            num_years = int(input("How many years of historical data do you want to add? "))
            for i in range(num_years):
                year = int(input(f"\nEnter year {i+1}: "))
                scope1 = float(input(f"Enter Scope 1 emissions for {year} (tCO2e): "))
                scope2 = float(input(f"Enter Scope 2 emissions for {year} (tCO2e): "))
                calculator.add_historical_data(year, scope1, scope2)
                print(f"Data for {year} added successfully.")
        except ValueError:
            print("Invalid input. Continuing without historical data.")

    while True:
        clear_screen()
        print(f"===== GHG Emissions Calculator - {company_name} =====")
        print("\nSelect an option:")
        print("1. Calculate new emissions")
        print("2. View historical emissions")
        print("3. Predict future emissions")
        print("4. Learn about GHG emissions")
        print("5. Exit")

        choice = input("\nEnter your choice (1-5): ")

        if choice == '1':
            clear_screen()
            print("===== Calculate New Emissions =====")

            print("\nSelect country/region for electricity emission factors:")
            countries = list(calculator.emission_factors['electricity'].keys())
            countries.remove('Custom')
            for i, country in enumerate(countries, 1):
                print(f"{i}. {country}")

            try:
                country_idx = int(input("\nEnter country/region number: ")) - 1
                country = countries[country_idx]

                print("\nEnter consumption data:")
                electricity = float(input("Electricity (kWh): "))
                diesel = float(input("Diesel (liters): "))
                petrol = float(input("Petrol (liters): "))
                fleet_diesel = float(input("Fleet Diesel (liters): "))
                fleet_petrol = float(input("Fleet Petrol (liters): "))

                use_optional = input("\nDo you want to enter optional consumption data? (y/n): ").lower() == 'y'
                cng = lpg = octane = 0

                if use_optional:
                    cng = float(input("CNG (m3): "))
                    lpg = float(input("LPG (m3): "))
                    octane = float(input("Octane (liters): "))

                scope1 = calculator.calculate_scope1(diesel, petrol, fleet_diesel, fleet_petrol, cng, lpg, octane)
                scope2 = calculator.calculate_scope2(electricity, country)
                total = scope1 + scope2

                print("\n===== Results =====")
                print(f"Scope 1 emissions: {scope1:.2f} tCO2e")
                print(f"Scope 2 emissions: {scope2:.2f} tCO2e")
                print(f"Total emissions: {total:.2f} tCO2e")

                save_data = input("\nDo you want to save this data to your historical records? (y/n): ").lower() == 'y'
                if save_data:
                    year = int(input("Enter the year for this data: "))
                    calculator.add_historical_data(year, scope1, scope2)
                    print(f"Data for {year} saved successfully.")

            except (ValueError, IndexError):
                print("Invalid input. Please try again.")

            input("\nPress Enter to continue...")

        elif choice == '2':
            clear_screen()
            print("===== Historical Emissions =====")

            data = calculator.get_historical_data()

            if data:
                print(f"\nHistorical emissions for {company_name}:")
                print("\nYear\tScope 1 (tCO2e)\tScope 2 (tCO2e)\tTotal (tCO2e)")
                print("--------------------------------------------------------")

                for year in sorted(data.keys()):
                    scope1 = data[year]['scope1']
                    scope2 = data[year]['scope2']
                    total = scope1 + scope2
                    print(f"{year}\t{scope1:.2f}\t\t{scope2:.2f}\t\t{total:.2f}")

                years = sorted(data.keys())
                if len(years) > 1:
                    print("\nYear-over-year changes:")
                    for i in range(1, len(years)):
                        prev_year = years[i - 1]
                        curr_year = years[i]

                        scope1_change = ((data[curr_year]['scope1'] - data[prev_year]['scope1']) / data[prev_year]['scope1']) * 100
                        scope2_change = ((data[curr_year]['scope2'] - data[prev_year]['scope2']) / data[prev_year]['scope2']) * 100
                        total_prev = data[prev_year]['scope1'] + data[prev_year]['scope2']
                        total_curr = data[curr_year]['scope1'] + data[curr_year]['scope2']
                        total_change = ((total_curr - total_prev) / total_prev) * 100

                        print(f"{prev_year} to {curr_year}:")
                        print(f"  Scope 1: {scope1_change:.2f}%")
                        print(f"  Scope 2: {scope2_change:.2f}%")
                        print(f"  Total: {total_change:.2f}%")
            else:
                print("No historical data available. Use option 1 to calculate and save emissions data.")

            input("\nPress Enter to continue...")

        elif choice == '3':
            clear_screen()
            print("===== Emissions Prediction =====")

            try:
                year = int(input("\nEnter prediction year: "))
                prediction = calculator.predict_emissions(year)

                if prediction:
                    scope1 = prediction['scope1']
                    scope2 = prediction['scope2']
                    total = scope1 + scope2

                    print(f"\nPredicted emissions for {company_name} in {year}:")
                    print(f"Scope 1: {scope1:.2f} tCO2e")
                    print(f"Scope 2: {scope2:.2f} tCO2e")
                    print(f"Total: {total:.2f} tCO2e")
                else:
                    print("Unable to predict emissions. Need at least two years of historical data.")
            except ValueError:
                print("Invalid input. Please try again.")

            input("\nPress Enter to continue...")

        elif choice == '4':
            clear_screen()
            print("===== Learn About GHG Emissions =====")
            print("\nScope 1 Emissions:")
            print("- Direct emissions from owned or controlled sources")
            print("- Examples: Fuel combustion, company vehicles, fugitive emissions")

            print("\nScope 2 Emissions:")
            print("- Indirect emissions from purchased electricity")
            print("- Examples: Purchased electricity, steam, heating, cooling")

            print("\nEmission Factors Used:")
            print("\nElectricity Emission Factors (tCO2e/kWh):")
            for country, factor in calculator.emission_factors['electricity'].items():
                if country != 'Custom':
                    print(f"- {country}: {factor}")

            print("\nFuel Emission Factors:")
            print(f"- Diesel: {calculator.emission_factors['diesel']} tCO2e/liter")
            print(f"- Petrol: {calculator.emission_factors['petrol']} tCO2e/liter")
            print(f"- CNG: {calculator.emission_factors['cng']} tCO2e/m³")
            print(f"- LPG: {calculator.emission_factors['lpg']} tCO2e/liter")
            print(f"- Octane: {calculator.emission_factors['octane']} tCO2e/m³")

            input("\nPress Enter to continue...")

        elif choice == '5':
            print("\nThank you for using the GHG Emissions Calculator!")
            break

if __name__ == "__main__":
    main()























===== GHG Emissions Calculator =====
Welcome to the Public GHG Emissions Calculator!
This tool helps you calculate and analyze Scope 1 and Scope 2 emissions.
Let's start by setting up your organization information.
